# GAME OF LIFE 

Nous sommes ravis de vous présenter notre premier projet en Python.

Le but de ce projet était de construire un Jeu de La Vie ou "Game of Life" afin de mettre en pratique nos connaissances. 

Nous avons travaillé dur pendant un mois et demi en respectant un ***plan de travail structuré*** qui vous serra présenté ci-dessous. 

Dans un premier temps, il a fallut mettre en place une ***organisation***. Cela nous a permit de respecter les deadlines tout en travaillant sur le projet de façons ***constante***. 
Ainsi, nous avons privilégier une première réunion de ***préconception*** pour éviter de nous lancer tête baissée dans le projet. 
Au cours de cette préconception nous nous sommes reparti les tâches selon les envies et les facilités de chacun. 
Nous avons également mis en place un document partagé et un Discord pour suivre les avancées du projet en continu. 
Chacun était libre de poser des questions et nous sommes resté soudé pour permettre au projet de voir le jour. 
De plus, nous avons choisi de mettre en place un ***projet GitHub*** afin de mettre en commun tous les dossiers crée. Cela a permit à chacun de tester le code et de suivre son avancement. 

Après avoir mis en place notre plan de travail nous avons commencé le codage. 
Il était necessaire pour nous de commencer par créer le jeu en console avant de l implémenter sous forme graphique. 

## Le jeu console ou Moteur de Jeu: 

Les cellules sont necessaires pour générer une grille. Nous avons décidés de créer une ***classe Cellule()*** pour pouvoir initialiser des cellules grâce à une même definition. 

### La Grille :
Pour cela on a créer une ***classe Grille()*** contenant des méthodes permettant son actualisation. 
    1- On doit définir pour chaque cellules ses cellules voisines et leur état = morte ou vivante. 

In [ ]:
#méthode pour définir les voisins de chaque cellule
    # return une liste des voisins de la cell 
    def find_voisin(self, cell_ligne, cell_colonne): 
        #créer une liste vide pour stocker les cellules voisines
        list_voisins = []
        # on parcour les cell en partant de la cellule a checker 
        # on parcourt entre:  -1 = cell précedante / 0 = même ligne ou colonne que la cell / 1 = cellule suivante
        for lignes in range(-1, 2): 
            for colonnes in range(-1,2): 
                voisin_ligne = cell_ligne + lignes
                voisin_colonne = cell_colonne + colonnes 

                #on vérifie que la cellule trouvé ne soit pas hors cadre ou que ça soit la même cellule
                voisin_OK = True 
                if (voisin_ligne) == cell_ligne and (voisin_colonne) == cell_colonne: 
                    voisin_OK = False
                if (voisin_ligne) < 0 or (voisin_colonne)>= self.colonnes: 
                    voisin_OK = False
                if (voisin_colonne) < 0 or (voisin_ligne) >= self.lignes: 
                    voisin_OK = False 
            
                if voisin_OK : 
                    list_voisins.append(self.grille2D[voisin_ligne][voisin_colonne])
        return list_voisin     

2 - Après avoir determiner si le nombre de cellules voisines de chaque cellule en stockant leur état on va finalement faire évoluer la grille en fonction des 3 règles du jeu. 
  # On créer de 2 listes pour stocker les cellules vivantes et mortes
  # on parcours la grille en modifiant l'état des cellulles en fonction des 3 règles 

In [ ]:
 
    def etat_suivant(self): 

        list_vivantes = []
        list_mortes = []

        for lignes in range (len(self.grille2D)): 
            for colonnes in range(len(self.grille2D[lignes])): 
                find_voisin = self.find_voisin(lignes, colonnes)
                #création d'une liste pour stocker le nb et les cell voisines vivantes
                nb_voisin_alive = []
                
                for cell_voisine in find_voisin : 
                    if cell_voisine.check_actual_state(): 
                        # on rempli la liste au fur et à mesure 
                        nb_voisin_alive.append(cell_voisine)

                cell = self.grille2D[lignes][colonnes]
                etat_cell = cell.check_actual_state()

                # on test les règles du jeu sur la cellule actuelle
                # on remplit à chaque fois la liste de cell morte ou vivante en fonction
                # si cell vivante alors : 
                if etat_cell == True : 
                    # reste vivante si elle a 2 ou 3 cell voisine vivantes 
                    if len(nb_voisin_alive) == 2 or len(nb_voisin_alive) == 3: 
                        list_vivantes.append(cell)
                    # meurt de sous ou de surpopulation
                    if len(nb_voisin_alive) < 2 or len(nb_voisin_alive) > 3:
                        list_mortes.append(cell)
                # si cell morte alors :
                else: 
                    # vit si exactement 3 cell voisines vivantes 
                    if len(nb_voisin_alive) == 3: 
                        list_vivantes.append(cell)
        # Enfin, on modifie l'état actuel de chaque cell analysée en parcourant les 2 listes créées    
        for cellule in list_vivantes: 
            cellule.cell_alive()
        for cellule in list_mortes:
            cellule.cell_dead() 

Une fois la grille fonctionnelle, nous allons maintenant nous concentrer sur l'affichage de notre jeu en utilisant les fonctions aupréalablement créer. 

In [ ]:
import pygame
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import time
import scipy.signal

# Game Engine

In [ ]:
class GameOfLife:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.board = np.random.choice([0, 1], size=(height, width), p=[0.7, 0.3])

    def save_to_file(self, filename):
        np.savetxt(filename, self.board, fmt="%d")

    def load_from_file(self, filename):
        self.board = np.loadtxt(filename, dtype=int)

    def update_board(self):
        kernel = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]])
        neighbors_count = scipy.signal.convolve2d(
            self.board, kernel, mode="same", boundary="fill", fillvalue=0
        )
        self.board = np.where(
            (self.board == 1) & ((neighbors_count < 2) | (neighbors_count > 3)),
            0,
            np.where(
                (self.board == 1) & ((neighbors_count == 2) | (neighbors_count == 3)),
                1,
                np.where((self.board == 0) & (neighbors_count == 3), 1, self.board),
            ),
        )
    

# Game Renderer

De plus, nous souhaitons définir et connaitre leur état à chaque instant. Cela nous permettra de générer des courbes et facilitera l évolution de la grille. 
On aura donc 3 méthodes : 


### La Grille : 
Nous avons opté pour une classe Grille () qui permettra de générer une grille avec des paramètres génériques tels que sa largeur et sa longueur (nombre de lignes / nombre de colonnes)
Il nous a paru également plus efficace d utiliser des méthodes pour créer, remplir, afficher et mettre à jour une grille de taille NxN. 
Ainsi, nous avons la définition de notre classe comme suit :  

### Evolution du jeu :
Pour se faire, les 3 règles suivantes doivent être respectées : 
- une cellule vivante survit si elle a 2 ou 3 voisins vivants.
- une cellule vivante meurt de solitude ou de surpopulation.
- une cellule morte peut devenir vivante si elle a exactement 3 voisins vivants.

Nous avons crée 2 méthodes pour respecter ces règles. 

#### Définir les voisins de chaque cellule dans la grille :  
* on crée un liste de cellules qui contiendra les cellules voisines et qui sera stocké dans les paramètres de la cellule. 
* on parcours chaque cellule de la grille en partant de la cellule à tester. 
* on parcours la liste en notant l état des cellules qu elle contient. 
* le nombre de cellules vivantes (= au voisines vivantes de la cellule à tester) sera stocké dans la cellule grâce au "self."

#### Faire évoluer la grille en respectant les 3 règles du jeu : 
   * on crée 2 listes vides pour stocker les cellules mortes et les cellules vivantes de la grille.
   * on recupère le nb de voisines vivantes de chaque cellule. 
   * on teste les 3 règles du jeu. 
   * on change l état de la cellule en fonction des règles. 
   * on affiche la nouvelle grille pour observer l évolution du jeu. 


In [ ]:
def update_board(self):
    kernel = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]])
    neighbors_count = scipy.signal.convolve2d(
        self.board, kernel, mode="same", boundary="fill", fillvalue=0
    )
    self.board = np.where(
        (self.board == 1) & ((neighbors_count < 2) | (neighbors_count > 3)),
        0,
        np.where(
            (self.board == 1) & ((neighbors_count == 2) | (neighbors_count == 3)),
            1,
            np.where((self.board == 0) & (neighbors_count == 3), 1, self.board),
        ),
    )

## 2- Le jeu en graphique: 

## 3- Analyse de l'évolution & courbes representatives: 

## 4- Le Lancement du jeu / Affichage: 
L utilisateur va lancer le jeu dans le fichier main.py 
A partir de là il aura plusieurs options : 
   * lancement en graphique avec affichage des courbes d'évolution
   * actualiser la grille manuellement ou non 
   * quitter la partie à tous moments 
Peut importe son choix il devra dans tous les cas entrer au clavier la taille de la grille souhaitée. 

***ATTENTION !*** 
Pour éviter un problème ou un bug nous devons ABSOLUEMENT vérifier que la taille entrée est bien un entier réel positif. 
Nous avons donc utilisé la ***méthode du blindage***. 

Il faudra aussi caster le nombre en entier car tout ce qui est entrée au clavier est forcément considéré par défaut comme un string (chaîne de caractère). 

Enfin, le jeu va évoluer petit à petit de façon plus ou moins autonome. 
Le joueur peut faire avancer le jeu grâce à un bouton (mode graphique) ou en tappant "q" sur son clavier (mode console).
Il est également possible de faire évoluer le jeu de manière autonome mais l évolution sera plus rapide et moins facile à analyser. 